In [67]:
import displayData
import pandas as p
import numpy as np
import matplotlib.pyplot as mp
import scipy.optimize as opt
from scipy.io import loadmat

In [68]:
#Data access
data = loadmat('ex4data1.mat')
weights = loadmat('ex4weights.mat')


y = data['y']
x = data['X']
y = np.matrix(y)
x = np.matrix(x)
y_T = y.T.ravel()
x_T = x.T
x = np.hstack((np.ones((np.shape(x)[0],1)),x))
x_T = x.T

m = x.shape[0]

weights = weights['Theta1'],weights['Theta2']
weights = np.matrix(weights)

In [209]:
def expand_labels(y, num_labels):
    y = (np.ones((np.shape(y)[0],num_labels)) * np.array(y))
    r = range(1,num_labels + 1)
    y = y == r
    y = y * 1
    y = np.matrix(y)
    return y

sigmoid_function = lambda a : 1 / (1 + np.vectorize(np.exp)(-a))

def forward_propagation(activation, weights, i, layers):
    if i == (layers - 2):
        return np.array(sigmoid_function(weights[0,i] @ activation))
    
    if i == -1:
        return forward_propagation(activation,weights,i+1,layers)
    else:
        sigmoid = sigmoid_function(weights[0,i] @ activation)
        activation = np.vstack((1,sigmoid))
        return forward_propagation(activation, weights, i+1, layers)

def backprop_cost(x_T,y,weights,layers,m,nlabels,l):
    h = []
    for i in np.arange(np.shape(x)[0]):
        h.append(forward_propagation(x_T[:,i],weights,-1,layers))
    h = np.array(h)
    h = np.squeeze(h)
    h_T = h.T

    y = expand_labels(y, nlabels)
    reg = (l/(2*m)) * (np.sum(np.sum(np.exp2(weights[0,0][:,1:]),1),0) + np.sum(np.sum(np.exp2(weights[0,1][:,1:]),1),0))
    cost = (1/m * np.sum((np.diag((-1 * y) @ np.log(h_T)) - np.diag((1 - y) @ np.log(1 - h_T))))) + reg
    
    return cost

def gradient_derivative(z):
    return sigmoid_function(z) @ (1 - sigmoid_function(z)).T

def random_weights(L_in, L_out):
    return np.random.uniform(-0.12,0.12,(L_out,1 + L_in))

def backpropagation_algorithm_3layer(param_nn, ninputs, layers , nlabels , x, y, reg):
   
    weights = random_weights(ninputs, param_nn[0]),random_weights(param_nn[0], param_nn[1])
    weights = np.matrix(weights)

    y = expand_labels(y, nlabels)
    gradient_1 = 0
    gradient_2 = 0

    x_T = x.T
    
    for i in np.arange(np.shape(x)[0]):
        
        ho = []
        hh = []
        
        ho.append(forward_propagation(x_T[:,i],weights,-1,layers))
        hh.append(forward_propagation(x_T[:,i],weights,-1,layers-1))
        ho = np.array(ho)
        ho = np.squeeze(ho)
        ho_T = ho.T

        hh = np.array(hh)
        hh = np.squeeze(hh)
        hh_T = hh.T
        print(hh)

        out_error = np.array(ho.T - np.asarray(y[i,:]).ravel())
        hide_error = (weights[0,1].T @ out_error.T) * gradient_derivative(hh)

        gradient_1 = gradient_1 + hide_error[1:] @ x_T[:,i]
        #gradient_2 = gradient_2 + .T @ x_T
        
        print(out_error)


In [210]:
backpropagation_algorithm_3layer([25,10], 400, 3 , 10 , x, y, 0.1)


[0.53234251 0.53561749 0.42266874 0.60363481 0.50761159 0.335997
 0.67288845 0.60400243 0.62908377 0.53847818 0.40793996 0.62553116
 0.37484591 0.66524328 0.48166528 0.51654718 0.43590132 0.73274874
 0.32829399 0.35351044 0.54548454 0.46898988 0.58578664 0.68469828
 0.57860505]


ValueError: shapes (25,) and (401,1) not aligned: 25 (dim 0) != 401 (dim 0)

In [50]:
a = np.matrix([[2,2],[1,1]])
np.sum(a,0)

matrix([[3, 3]])